**PART 1**

In [7]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
drive.mount('/content/drive')
default_path = '/content/drive/My Drive/INF ML 552/HW7 - HMM/'

grid = []
towers = []
noisy_distances = []

file = open(default_path+"hmm-data.txt")
file.readline()
file.readline()
for i in range(10):
  grid.append([])
  grid[i].extend([int(x) for x in file.readline().split()])

[file.readline() for _ in range(4)]

for i in range(4):
  towers.append([])
  towers[i].extend([int(x) for x in file.readline().split(' ')[2:]])

[file.readline() for _ in range(4)]

for i in range(11):
  noisy_distances.append([])
  noisy_distances[i].extend([float(x) for x in file.readline().split()])

print(len(noisy_distances))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
11


In [0]:
def get_valid_states(grid):

  valid_states = np.ones(len(grid)*len(grid[0]), dtype=bool)
  states = {}

  for i, row in enumerate(grid):
    for j, col in enumerate(grid[i]):
      states[i*10+j] = (i,j)
      if col == 0:
          valid_states[i*10+j] = False

  # print(states)
  return states, valid_states

# get_valid_states(grid)

In [0]:
def get_intitial_prob(grid):
  s = 0
  for row in grid:
    s += sum(row)
  
  initial_prob = np.ones(len(grid)*len(grid[0])) * (1/s)
  for i, row in enumerate(grid):
    for j, col in enumerate(grid[i]):
      # print(i,j,row,col)
      if col == 0:
        initial_prob[i*10+j] = 0
      
  return initial_prob

# get_intitial_prob(grid)

In [0]:
def get_prob(grid, states, valid_states):
  prob = []
  for i, st in states.items():
    if valid_states[i]:
      neighbors = 0
      x, y = st
      if 0<=x-1<=9 and grid[x-1][y] == 1:
        neighbors += 1
      if 0<=x+1<=9 and grid[x+1][y] == 1:
        neighbors +=1
      if 0<=y-1<=9 and grid[x][y-1] == 1:
        neighbors +=1
      if 0<=y+1<=9 and grid[x][y+1] == 1:
        neighbors +=1
      prob.append(1/neighbors)
    else:
      prob.append(0)
  # print(prob)
  return prob

In [0]:
def get_cpt(grid, states, valid_states):

  tran_matrix = []
  prob = get_prob(grid,states,valid_states)
  # print(grid)
  # print(states)
  # print(valid_states)
  for i, st in states.items():
    if valid_states[i]:
      from_x, from_y = st
      a = [list(np.zeros(len(k))) for k in grid]
      for j, st1 in states.items():
        to_x, to_y = st1
        if ((from_y == to_y and (from_x == to_x - 1 or from_x == to_x + 1)) or  (from_x == to_x and (from_y == to_y - 1 or from_y == to_y + 1))) and valid_states[j]:
          a[to_x][to_y] = prob[i]

      a = [item for sublist in a for item in sublist]
      tran_matrix.append(a)
    else:
      tran_matrix.append(list(np.zeros(len(valid_states))))
  # pd.DataFrame(tran_matrix).to_csv(default_path + "data.csv")
  # return np.transpose(np.array(tran_matrix)).tolist()
  return tran_matrix   



In [0]:
import math 

def get_distance(states, valid_states, grid, towers):

  possible_observations = []
  euclidean_dist_matrix = []

  # print(round(math.sqrt(2 * ((len(grid)-1)**2)),1))
  for i in range(0, int(round(max_noise * math.sqrt(2 * ((len(grid)-1)**2)),1)/0.1)+1):
    possible_observations.append(round(i*0.1,1))

  for t, tower in enumerate(towers):
    euclidean_dist_matrix.append([])
    tx = tower[0]
    ty = tower[1]
    st_obs = []
    for st in states:
      # print(st)
      st_obs.append([])
      if valid_states[st]:
        x, y = states[st]
        min_dist = round(min_noise * math.sqrt((x-tx)**2 + (y-ty)**2) , 1)
        max_dist = round(max_noise * math.sqrt((x-tx)**2 + (y-ty)**2) , 1)
        # print("dist ",min_dist, max_dist)
        p = 1/(((max_dist - min_dist + 0.1) / 0.1)) if (tx,ty) != (x,y) else 0.0
        # print("prob ", p)
        # st_obs.extend([0.0 for x in range(0, int(min_dist/0.1)+1)])
        # st_obs.extend([p for x in range(0, int((max_dist - min_dist) / 0.1)+1)])
        # st_obs.extend([0.0 for x in range(0, len(possible_observations)- len(st_obs))])
        for i in possible_observations:
          if (min_dist <= i <= max_dist):
              st_obs[st].extend([p])
          else:
              st_obs[st].extend([0])

      else:
        for i in possible_observations:
          st_obs[st].extend([0])
    euclidean_dist_matrix[t].extend(st_obs)
  
  # print(np.array(euclidean_dist_matrix).shape)
  return possible_observations, euclidean_dist_matrix

# get_distance(states, valid_states, grid, towers)

In [0]:
def get_evidence_matrix(evidence, states, valid_states, possible_obs, ed_matrix): #observation_seq, states, observations, emission_prob

  prob = np.ones((1, len(states))) #state_prob
  p = []
  _emission_m = []

  # print(np.array(ed_matrix).shape)
  
  for i, given_tower_noise in enumerate(evidence):
      # print("noise", given_tower_noise)
      p.append([])
      for j, state in enumerate(states):
        if valid_states[j]:
          for k, obs in enumerate(possible_obs): 
              if(given_tower_noise == obs):
                  # print(i,j,k,given_tower_noise, obs)
                  p[i].extend([ed_matrix[i][j][k]])
                  break
        else:
          p[i].extend([0.0])
  
  for i in p:
    # print(i)
    prob *= i 
  
  # pd.concat([pd.DataFrame(p), pd.DataFrame(prob)]).to_csv(default_path+'data.csv')
  return prob



In [0]:
def HMM(t1, states, valid_states, tran_matrix, possible_observations, euclidean_dist_matrix, noisy_distances): #observation_seq

  evidence_prob = [] #_em_prob
  prev_states = [] #_previous_states
  seq = [] #_states_seq
  back_seq = []
  max_prob = []

  for step in range(len(noisy_distances)):
    # print("step", step)
    evidence_prob = get_evidence_matrix(noisy_distances[step], states, valid_states, possible_observations, euclidean_dist_matrix)
    # print(evidence_prob)
    # return
    if step == 0:
      max_prob = t1 * evidence_prob         
    else:
        # max_prob, prev_states = max_prob_state(max_prob * tran_matrix) 
        states_prob = max_prob * tran_matrix
        p = 0
        prev_state_max = 0
        max_prob = []
        prev_states = []
        
        for curr_state in range(len(states)):
          max_prob_val = 0
          pr_st = 0
          if valid_states[curr_state]:
            for pr_st in range(len(states)):
              if valid_states[pr_st]:
                p = states_prob[curr_state][pr_st]
                if (p > max_prob_val):
                  max_prob_val = p
                  prev_state_max = pr_st
            max_prob.extend([max_prob_val]) 
            # print(curr_state, ":", prev_state_max ,":", max_prob_val) 
            prev_states.extend([prev_state_max])
          else:
            max_prob.extend([0])
            prev_states.extend([None]) 

        max_prob = max_prob * evidence_prob
        back_seq.append(prev_states)

  # print(np.array(max_prob).shape, max_prob[0],max_prob[0].tolist().index(max(max_prob[0].tolist())))  
  seq.extend([max_prob[0].tolist().index(max(max_prob[0].tolist()))])
  back_seq.reverse()
  for state in back_seq:
      seq.extend([state[seq[-1]]])
  seq.reverse()    
  return seq


In [0]:
min_noise = 0.7
max_noise = 1.3
t1 = get_intitial_prob(grid)
states, valid_states = get_valid_states(grid)
tran_matrix = get_cpt(grid, states, valid_states)
possible_observations, euclidean_dist_matrix = get_distance(states, valid_states, grid, towers)
seq = HMM(t1, states, valid_states, tran_matrix, possible_observations, euclidean_dist_matrix, noisy_distances)
for t, i in enumerate(seq):
  print("Time ", t+1, ": " ,states[i])

PART 2

In [34]:
pip install hidden_markov

  Created wheel for hidden-markov: filename=hidden_markov-0.3.2-cp36-none-any.whl size=6906 sha256=4a0fa2de1ae12231136b1206683a87e8507182dcff2a31bd738b221200a781e4
  Stored in directory: /root/.cache/pip/wheels/a1/e1/d8/7d0e168c63377453769325485b214bf6ca2ce5e5cf0f67a5ae
Successfully built hidden-markov


In [0]:
import hidden_markov

min_noise = 0.7
max_noise = 1.3

states = []
for i in range(10):
  for j in range(10):
    if grid[i][j] == 1:
      states.append((i,j))
# states = set(states)

possible_observations = []
for i in range(0, int(round(max_noise * math.sqrt(2 * ((len(grid)-1)**2)),1)/0.1)+1):
    possible_observations.append(round(i*0.1,1))

# possible_observations = set(possible_observations)

start_probability = [[1/len(states)] for i in range(len(states))]
start_probability = np.matrix(start_probability)

tran_matrix = []
prob = []
for i, st in enumerate(states):
    neighbors = 0
    x, y = st
    if 0<=x-1<=9 and grid[x-1][y] == 1:
      neighbors += 1
    if 0<=x+1<=9 and grid[x+1][y] == 1:
      neighbors +=1
    if 0<=y-1<=9 and grid[x][y-1] == 1:
      neighbors +=1
    if 0<=y+1<=9 and grid[x][y+1] == 1:
      neighbors +=1
    prob.append(1/neighbors)
# print(grid)
# print(states)
# print(valid_states)
for i, st in enumerate(states):
  # if valid_states[i]:
  from_x, from_y = st
  a = list(np.zeros(len(states)))
  for j, st1 in enumerate(states):
    to_x, to_y = st1
    if ((from_y == to_y and (from_x == to_x - 1 or from_x == to_x + 1)) or  (from_x == to_x and (from_y == to_y - 1 or from_y == to_y + 1))):
      a[j] = prob[i]
  tran_matrix.append(a)

transition_probability = np.matrix(tran_matrix)

emission_probability = []

for i, st in enumerate(states):

  obs = []
  x, y = st
  for j , tower in enumerate(towers):
    t = []
    tx, ty = tower
    # print(tx,ty,x,y)
    min_dist = round(min_noise * math.sqrt((x-tx)**2 + (y-ty)**2) , 1)
    max_dist = round(max_noise * math.sqrt((x-tx)**2 + (y-ty)**2) , 1)
    p = 1/(((max_dist - min_dist + 0.1) / 0.1)) if (tx,ty) != (x,y) else 0.0
    for k,val in enumerate(possible_observations):
      if min_dist <= val <= max_dist:
        t.append(p)
      else:
        t.append(0)
    obs.append(t)

    final = np.ones(len(possible_observations))
    for i in obs:
      final *= i
  
  emission_probability.append(final)

emission_probability = np.matrix(emission_probability)
print(emission_probability.shape)

#emission_probbablity is 3D while library accepts only 2D matrix.
test = hidden_markov.hmm(states,possible_observations,start_probability,transition_probability,emission_probability)
print(test.viterbi(noisy_distances))